<a href="https://colab.research.google.com/github/salmon-Xin/BERT-for-question-answering-on-DRCD-dataset/blob/main/%E3%80%8CBERT_for_question_answering_on_DRCD_dataset%E3%80%8D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers datasets accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!nvidia-smi

shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
Sun Dec  4 17:35:16 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P0    30W /  70W |  12118MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+---------

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%cd /content/drive/"MyDrive"/"Colab Notebooks"

/content/drive/MyDrive/Colab Notebooks


In [ ]:
!ls

shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
ls: cannot open directory '.': Transport endpoint is not connected


In [ ]:
# !mkdir ./BERT_DRCD/
%cd ./BERT_DRCD/

/content/drive/MyDrive/Colab Notebooks/BERT_DRCD


# Delta Reading Comprehension Dataset 
台達閱讀理解資料集 Delta Reading Comprehension Dataset (DRCD) 屬於通用領域繁體中文機器閱讀理解資料集。
本資料集期望成為適用於遷移學習之標準中文閱讀理解資料集。
本資料集從2,108篇維基條目中整理出10,014篇段落，並從段落中標註出30,000多個問題

關於資料集之更詳細資訊請洽詢論文：
For more information please refer to Paper https://arxiv.org/abs/1806.00920

## Data format 資料格式

- version : <String> 資料集版本
- data : <Array>
  - title : <String> : 文章標題
  - id : <String> : 文章編號
  - paragraphs : <Array>
    - id : <String> : 文章編號_段落編號
    - context : <String> : 段落內容
    - qas : <Array>
      - question : <String> : 問題內容
      - id :<String> : 文章編號_段落編號_問題編號
      - answers : <Arrays>
        - answer_start : <int> text在文中位置
        - id : <String> : "1"表示為人工標註的答案，"2"以上為人工答題的答案
        - text : <string> : 答案內容

**References:**

- [BERT](https://arxiv.org/pdf/1810.04805.pdf)
- [DRCD](https://arxiv.org/abs/1806.00920)

In [ ]:
from tensorflow import keras
import json
from pprint import pprint

# Download the data
train_data_url = "https://raw.githubusercontent.com/DRCKnowledgeTeam/DRCD/master/DRCD_training.json"
train_path = keras.utils.get_file("train.json", train_data_url)
eval_data_url = "https://github.com/DRCKnowledgeTeam/DRCD/blob/master/DRCD_dev.json?raw=true"
eval_path = keras.utils.get_file("eval.json", eval_data_url)

with open(train_path, "r", encoding="UTF-8") as f:
    raw_train_data = json.load(f)

with open(eval_path, "r", encoding="UTF-8") as f:
    raw_eval_data = json.load(f)

for ele in raw_train_data['data']:
  pprint(ele['paragraphs'][0])
  break
  



2184379/2184379 [==============================] - 0s 0us/step
{'context': '2010年引進的廣州快速公交運輸系統，屬世界第二大快速公交系統，日常載客量可達100萬人次，高峰時期每小時單向客流高達26900人次，僅次於波哥大的快速交通系統，平均每10秒鐘就有一輛巴士，每輛巴士單向行駛350小時。包括橋樑在內的站台是世界最長的州快速公交運輸系統站台，長達260米。目前廣州市區的計程車和公共汽車主要使用液化石油氣作燃料，部分公共汽車更使用油電、氣電混合動力技術。2012年底開始投放液化天然氣燃料的公共汽車，2014年6月開始投放液化天然氣插電式混合動力公共汽車，以取代液化石油氣公共汽車。2007年1月16日，廣州市政府全面禁止在市區內駕駛摩托車。違反禁令的機動車將會予以沒收。廣州市交通局聲稱禁令的施行，使得交通擁擠問題和車禍大幅減少。廣州白雲國際機場位於白雲區與花都區交界，2004年8月5日正式投入運營，屬中國交通情況第二繁忙的機場。該機場取代了原先位於市中心的無法滿足日益增長航空需求的舊機場。目前機場有三條飛機跑道，成為國內第三個擁有三跑道的民航機場。比鄰近的香港國際機場第三跑道預計的2023年落成早8年。',
 'id': '1001-10',
 'qas': [{'answers': [{'answer_start': 84, 'id': '1', 'text': '10秒鐘'}],
          'id': '1001-10-1',
          'question': '廣州的快速公交運輸系統每多久就會有一輛巴士？'},
         {'answers': [{'answer_start': 256, 'id': '1', 'text': '2007年1月16日'}],
          'id': '1001-10-2',
          'question': '從哪一天開始在廣州市內騎摩托車會被沒收？'},
         {'answers': [{'answer_start': 447, 'id': '1', 'text': '香港國際機場'}],
          'id': '1001-10-3',
          'question': '

讀取資料

In [ ]:
from pathlib import Path
def read_data(path, limit=None):
    path = Path(path)
    with open(path, 'rb') as f:
        data_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
    for group in data_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                # answer = qa['answers']
                for answer in qa['answers']:
                  contexts.append(context)
                  questions.append(question)
                  answers.append(answer)
                if limit != None and len(contexts) > limit:
                  return contexts, questions, answers
                  
    return contexts, questions, answers

In [ ]:
train_contexts, train_questions, train_answers = read_data(train_path, 10000)
eval_contexts, eval_questions, eval_answers = read_data(eval_path,2000)

In [ ]:
print(len(train_contexts))
print(len(eval_contexts))

10001
2002


In [ ]:
print('train_data')
print("context:",train_contexts[0])
print("question:",train_questions[0])
print("answer:",train_answers[0])

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



train_data
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-79-21955d71c0b4>", line 2, in <module>
    print("context:",train_contexts[0])
NameError: name 'train_contexts' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 2040, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python3.8/dist-p

NameError: ignored

In [ ]:
print("evaluation_data")
print("context:",eval_contexts[0])
print("question:",eval_questions[0])
print("answer:",eval_answers[0])

evaluation_data
context: 在歐洲，梵語的學術研究，由德國學者陸特和漢斯雷頓開創。後來威廉·瓊斯發現印歐語系，也要歸功於對梵語的研究。此外，梵語研究，也對西方文字學及歷史語言學的發展，貢獻不少。1786年2月2日，亞洲協會在加爾各答舉行。會中，威廉·瓊斯發表了下面這段著名的言論：「梵語儘管非常古老，構造卻精妙絕倫：比希臘語還完美，比拉丁語還豐富，精緻之處同時勝過此兩者，但在動詞詞根和語法形式上，又跟此兩者無比相似，不可能是巧合的結果。這三種語言太相似了，使任何同時稽考三者的語文學家都不得不相信三者同出一源，出自一種可能已經消逝的語言。基於相似的原因，儘管缺少同樣有力的證據，我們可以推想哥德語和凱爾特語，雖然混入了迥然不同的語彙，也與梵語有著相同的起源；而古波斯語可能也是這一語系的子裔。」
question: 陸特和漢斯雷頓開創了哪一地區對梵語的學術研究？
answer: {'id': '1', 'text': '歐洲', 'answer_start': 1}


In [ ]:
print("context:",eval_contexts[21])
print("question:",eval_questions[21])
print("answer:",eval_answers[21])

context: 馬祖列島是隸屬中華民國的群島，位於臺灣海峽正北方，面臨閩江口、連江口和羅源灣，與中國大陸僅一水之隔，距中國大陸最近點約9.25公里。主要由南竿島、北竿島、高登島、亮島、東莒島、西莒島、東引島、西引島及其附屬小島共計36個島嶼、礁嶼組成，面積29.6平方公里，居民人口約12,600多人。1999年以馬祖列島為範圍的馬祖國家風景區成立，隸屬於交通部觀光局管理。除了地形地貌和人文特色外，因地理位置關係而成為多種候鳥過境或渡冬的區域，2000年成立馬祖列島燕鷗保護區，主要針對白眉燕鷗、紅燕鷗、蒼燕鷗、鳳頭燕鷗、黑尾鷗、岩鷺、插尾雨燕等7種鳥類。2000年6月曾在此發現在世界鳥類紅皮書中被列為臨絕種的黑嘴端鳳頭燕鷗共4對，每年的最佳賞燕鷗季為5月到8月。馬祖除通用國語之外，還通用操長樂口音的閩東語，當地稱為「平話」、「馬祖話」或「福州話」，官方則稱之為「福州語」。
question: 負責管理馬祖國家風景區的單位為？
answer: {'id': '2', 'text': '交通部觀光局', 'answer_start': 170}


In [ ]:
# 新增 answer 的結束位置
def add_end_idx(answers):
    for answer in answers:
        gold_text = answer['text']
        start_idx = answer['answer_start']
        if gold_text == '':
          end_idx = 0
        else:
          end_idx = start_idx + len(gold_text) # Find end character index of answer in context
        answer['answer_end'] = end_idx

add_end_idx(train_answers)
add_end_idx(eval_answers)

In [ ]:
print(eval_answers[0])
print(eval_answers[21])

{'id': '1', 'text': '歐洲', 'answer_start': 1, 'answer_end': 3}
{'id': '2', 'text': '交通部觀光局', 'answer_start': 170, 'answer_end': 176}


將資料進行 Tokenize


將 input 資料轉換成 token id 、tpye_id 與 attention_mask

In [ ]:
from transformers import AutoTokenizer, BertTokenizerFast, AutoConfig
tokenizer = AutoTokenizer.from_pretrained('bert-base-chinese')
tokenizer_fast = BertTokenizerFast.from_pretrained('bert-base-chinese')
config = AutoConfig.from_pretrained('bert-base-chinese')

train_encodings = tokenizer_fast(train_contexts, train_questions, truncation=True, padding=True)
eval_encodings = tokenizer_fast(eval_contexts, eval_questions, truncation=True, padding=True)

NameError: ignored

In [ ]:
print('tokenizer 最大input長度 = ' ,tokenizer.model_max_length)

tokenizer 最大input長度 =  512


In [ ]:
train_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
# use AutoTokenizer
print("input_ids\n" ,train_encodings['input_ids'][0])
print("length = ",len(train_encodings['input_ids'][0]))
print("token_type_ids = ",train_encodings['token_type_ids'][0])
print("attention_mask = ",train_encodings['attention_mask'][0])

print("\ninput_ids convert_ids_to_tokens \n",tokenizer.convert_ids_to_tokens(train_encodings['input_ids'][0]))
print("\ninput_ids decode\n", tokenizer.decode(train_encodings['input_ids'][0]))

print("\nattention_mask\n",train_encodings['attention_mask'][0])
print("length = ",len(train_encodings['attention_mask'][0]))


input_ids
 [101, 8166, 2399, 2471, 6868, 4638, 2451, 2336, 2571, 6862, 1062, 769, 6880, 6745, 5143, 5186, 8024, 2253, 686, 4518, 5018, 753, 1920, 2571, 6862, 1062, 769, 5143, 5186, 8024, 3189, 2382, 6734, 2145, 7030, 1377, 6888, 8135, 5857, 782, 3613, 8024, 7770, 2292, 3229, 3309, 3680, 2207, 3229, 1606, 1403, 2145, 3837, 7770, 6888, 11023, 8279, 782, 3613, 8024, 1006, 3613, 3176, 3797, 1520, 1920, 4638, 2571, 6862, 769, 6858, 5143, 5186, 8024, 2398, 1772, 3680, 8108, 4907, 7132, 2218, 3300, 671, 6739, 2349, 1894, 8024, 3680, 6739, 2349, 1894, 1606, 1403, 6121, 7691, 8612, 2207, 3229, 511, 1259, 2886, 3578, 3558, 1762, 1058, 4638, 4991, 1378, 3221, 686, 4518, 3297, 7269, 4638, 2336, 2571, 6862, 1062, 769, 6880, 6745, 5143, 5186, 4991, 1378, 8024, 7269, 6888, 9044, 5101, 511, 4680, 1184, 2451, 2336, 2356, 1281, 4638, 6243, 4923, 6722, 1469, 1062, 1066, 3749, 6722, 712, 6206, 886, 4500, 3890, 1265, 4767, 3779, 3706, 868, 4234, 3160, 8024, 6956, 1146, 1062, 1066, 3749, 6722, 3291, 886, 45

In [ ]:
# use BertTokenizerFast ######
print("input_ids\n" ,train_encodings['input_ids'][0])
print("length = ",len(train_encodings['input_ids'][0]))
print("token_type_ids = ",train_encodings['token_type_ids'][0])
print("attention_mask = ",train_encodings['attention_mask'][0])

print("\ninput_ids convert_ids_to_tokens \n",tokenizer_fast.convert_ids_to_tokens(train_encodings['input_ids'][0]))
print("\ninput_ids decode\n", tokenizer_fast.decode(train_encodings['input_ids'][0]))

print("\nattention_mask\n",train_encodings['attention_mask'][0])
print("length = ",len(train_encodings['attention_mask'][0]))

input_ids
 [101, 8166, 2399, 2471, 6868, 4638, 2451, 2336, 2571, 6862, 1062, 769, 6880, 6745, 5143, 5186, 8024, 2253, 686, 4518, 5018, 753, 1920, 2571, 6862, 1062, 769, 5143, 5186, 8024, 3189, 2382, 6734, 2145, 7030, 1377, 6888, 8135, 5857, 782, 3613, 8024, 7770, 2292, 3229, 3309, 3680, 2207, 3229, 1606, 1403, 2145, 3837, 7770, 6888, 11023, 8279, 782, 3613, 8024, 1006, 3613, 3176, 3797, 1520, 1920, 4638, 2571, 6862, 769, 6858, 5143, 5186, 8024, 2398, 1772, 3680, 8108, 4907, 7132, 2218, 3300, 671, 6739, 2349, 1894, 8024, 3680, 6739, 2349, 1894, 1606, 1403, 6121, 7691, 8612, 2207, 3229, 511, 1259, 2886, 3578, 3558, 1762, 1058, 4638, 4991, 1378, 3221, 686, 4518, 3297, 7269, 4638, 2336, 2571, 6862, 1062, 769, 6880, 6745, 5143, 5186, 4991, 1378, 8024, 7269, 6888, 9044, 5101, 511, 4680, 1184, 2451, 2336, 2356, 1281, 4638, 6243, 4923, 6722, 1469, 1062, 1066, 3749, 6722, 712, 6206, 886, 4500, 3890, 1265, 4767, 3779, 3706, 868, 4234, 3160, 8024, 6956, 1146, 1062, 1066, 3749, 6722, 3291, 886, 45

In [ ]:
# 新增答案的起始結束位置
def add_token_positions(encodings,answers):
  start_positions = []
  end_positions = []
  for i in range(len(answers)):
    if answers[i]['answer_start'] == 0 and answers[i]['answer_end'] == 0:
      start_positions.append(0)
      end_positions.append(0)
    else:
      start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
      end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))
    # if None, the answer passage has been truncated
    if start_positions[-1] is None:
      start_positions[-1] = tokenizer.model_max_length
    if end_positions[-1] is None:
      end_positions[-1] = tokenizer.model_max_length
  encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

In [ ]:
print(len(train_answers))
print(len(eval_answers))
add_token_positions(train_encodings, train_answers)
add_token_positions(eval_encodings, eval_answers)

10001
2002


定義dataset並轉換成tensor格式

In [ ]:
# 定義dataset並轉換成tensor格式
import torch
class Dataset(torch.utils.data.Dataset):
  def __init__(self, encodings):
    self.encodings = encodings

  def __getitem__(self, idx):
    return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

  def __len__(self):
    return len(self.encodings.input_ids)

In [ ]:
train_dataset = Dataset(train_encodings)
eval_dataset = Dataset(eval_encodings)

In [ ]:
train_dataset[0]

{'input_ids': tensor([  101,  8166,  2399,  2471,  6868,  4638,  2451,  2336,  2571,  6862,
          1062,   769,  6880,  6745,  5143,  5186,  8024,  2253,   686,  4518,
          5018,   753,  1920,  2571,  6862,  1062,   769,  5143,  5186,  8024,
          3189,  2382,  6734,  2145,  7030,  1377,  6888,  8135,  5857,   782,
          3613,  8024,  7770,  2292,  3229,  3309,  3680,  2207,  3229,  1606,
          1403,  2145,  3837,  7770,  6888, 11023,  8279,   782,  3613,  8024,
          1006,  3613,  3176,  3797,  1520,  1920,  4638,  2571,  6862,   769,
          6858,  5143,  5186,  8024,  2398,  1772,  3680,  8108,  4907,  7132,
          2218,  3300,   671,  6739,  2349,  1894,  8024,  3680,  6739,  2349,
          1894,  1606,  1403,  6121,  7691,  8612,  2207,  3229,   511,  1259,
          2886,  3578,  3558,  1762,  1058,  4638,  4991,  1378,  3221,   686,
          4518,  3297,  7269,  4638,  2336,  2571,  6862,  1062,   769,  6880,
          6745,  5143,  5186,  4991,  1

In [ ]:
len(train_dataset)

10001

# 載入模型架構( BertForQuestionAnswering )

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoConfig , BertForQuestionAnswering
config = AutoConfig.from_pretrained('bert-base-chinese')
model = BertForQuestionAnswering.from_pretrained('bert-base-chinese', config=config)

Downloading:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-chinese a

In [ ]:
print(model)

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

########訓練模型###########

In [ ]:
import logging
import datasets
from datasets import load_dataset, load_metric
from torch.utils.data import DataLoader
from tqdm.auto import tqdm, trange
import math

import transformers
from accelerate import Accelerator
from transformers import (
    AdamW,
    AutoConfig,
    default_data_collator,
    get_scheduler
)

In [ ]:
train_batch_size = 2      # 設定 training batch size
eval_batch_size = 2     # 設定 eval batch size
num_train_epochs = 3      # 設定 epoch

將資料丟入 DataLoader

## Optimizer 、Learning rate 、Scheduler 設定

In [ ]:
data_collator = default_data_collator
train_dataloader = DataLoader(train_dataset, shuffle=True, collate_fn=data_collator, batch_size=train_batch_size)
eval_dataloader = DataLoader(eval_dataset, collate_fn=data_collator, batch_size=eval_batch_size)

In [ ]:
learning_rate=3e-5          # 設定 learning_rate
gradient_accumulation_steps = 1   # 設定 幾步後進行反向傳播

no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },                                
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)

# Scheduler and math around the number of training steps.
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / gradient_accumulation_steps)
max_train_steps = num_train_epochs * num_update_steps_per_epoch
print('max_train_steps', max_train_steps)

# scheduler
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=max_train_steps,
)

max_train_steps 15003


/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


## 將資料、參數丟入 Accelerator

> 縮排段落





In [ ]:
# Initialize the accelerator. We will let the accelerator handle device placement for us in this example.
accelerator = Accelerator()

# Prepare everything with our `accelerator`.
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [ ]:
!nvidia-smi

Sat Dec  3 15:01:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    26W /  70W |   1032MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

開始訓練

In [ ]:
# Train!
logger = logging.getLogger(__name__)
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger.info(accelerator.state)
output_dir = 'model/'


total_batch_size = train_batch_size * accelerator.num_processes * gradient_accumulation_steps

logger.info("***** Running training *****")
logger.info(f"  Num examples = {len(train_dataset)}")
logger.info(f"  Num Epochs = {num_train_epochs}")
logger.info(f"  Instantaneous batch size per device = {train_batch_size}")
logger.info(f"  Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
logger.info(f"  Gradient Accumulation steps = {gradient_accumulation_steps}")
logger.info(f"  Total optimization steps = {max_train_steps}")


completed_steps = 0
best_epoch = {"epoch:": 0, "acc": 0 }

for epoch in trange(num_train_epochs, desc="Epoch"):
  model.train()
  for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
    
    outputs = model(**batch)
    loss = outputs.loss
    loss = loss / gradient_accumulation_steps
    accelerator.backward(loss)
    if step % gradient_accumulation_steps == 0 or step == len(train_dataloader) - 1:
      optimizer.step()
      lr_scheduler.step()
      optimizer.zero_grad()
      completed_steps += 1

    if step % 50 == 0:
      print({'epoch': epoch, 'step': step, 'loss': loss.item()})

    if completed_steps >= max_train_steps:
      break
      
  # logger.info("***** Running eval *****")
  # model.eval()
  # for step, batch in enumerate(tqdm(eval_dataloader, desc="Eval Iteration")):
  #   outputs = model(**batch)
  #   predictions = outputs.logits.argmax(dim=-1)
  #   metric.add_batch(
  #       predictions=accelerator.gather(predictions),
  #       references=accelerator.gather(batch["labels"]),
  #   )

  # eval_metric = metric.compute()
  # logger.info(f"epoch {epoch}: {eval_metric}")
  # if eval_metric > best_epoch['acc']:
  #   best_epoch['epoch'] = num_train_epochs
  #   best_epoch['acc'] = eval_metric


  if output_dir is not None:
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir + 'new_epoch_' + str(epoch) + '/', save_function=accelerator.save)


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5001 [00:00<?, ?it/s]

{'epoch': 0, 'step': 0, 'loss': 6.3488569259643555}
{'epoch': 0, 'step': 50, 'loss': 3.1839828491210938}
{'epoch': 0, 'step': 100, 'loss': 2.427407741546631}
{'epoch': 0, 'step': 150, 'loss': 2.059192419052124}
{'epoch': 0, 'step': 200, 'loss': 3.244138240814209}
{'epoch': 0, 'step': 250, 'loss': 1.5916250944137573}
{'epoch': 0, 'step': 300, 'loss': 1.5804007053375244}
{'epoch': 0, 'step': 350, 'loss': 2.5286612510681152}
{'epoch': 0, 'step': 400, 'loss': 0.5927287936210632}
{'epoch': 0, 'step': 450, 'loss': 1.0758029222488403}
{'epoch': 0, 'step': 500, 'loss': 0.38823938369750977}
{'epoch': 0, 'step': 550, 'loss': 5.235383987426758}
{'epoch': 0, 'step': 600, 'loss': 1.0152223110198975}
{'epoch': 0, 'step': 650, 'loss': 0.5352976322174072}
{'epoch': 0, 'step': 700, 'loss': 3.270167350769043}
{'epoch': 0, 'step': 750, 'loss': 1.5750904083251953}
{'epoch': 0, 'step': 800, 'loss': 2.4815306663513184}
{'epoch': 0, 'step': 850, 'loss': 1.8956583738327026}
{'epoch': 0, 'step': 900, 'loss': 1

Iteration:   0%|          | 0/5001 [00:00<?, ?it/s]

{'epoch': 1, 'step': 0, 'loss': 0.1614847481250763}
{'epoch': 1, 'step': 50, 'loss': 0.35084760189056396}
{'epoch': 1, 'step': 100, 'loss': 1.3423324823379517}
{'epoch': 1, 'step': 150, 'loss': 0.0944417342543602}
{'epoch': 1, 'step': 200, 'loss': 0.2067795842885971}
{'epoch': 1, 'step': 250, 'loss': 0.01170733105391264}
{'epoch': 1, 'step': 300, 'loss': 0.16723987460136414}
{'epoch': 1, 'step': 350, 'loss': 0.07557608932256699}
{'epoch': 1, 'step': 400, 'loss': 0.484496533870697}
{'epoch': 1, 'step': 450, 'loss': 0.015415122732520103}
{'epoch': 1, 'step': 500, 'loss': 0.07821211218833923}
{'epoch': 1, 'step': 550, 'loss': 0.014474182389676571}
{'epoch': 1, 'step': 600, 'loss': 0.03631230443716049}
{'epoch': 1, 'step': 650, 'loss': 0.027333594858646393}
{'epoch': 1, 'step': 700, 'loss': 1.499333143234253}
{'epoch': 1, 'step': 750, 'loss': 0.11334466189146042}
{'epoch': 1, 'step': 800, 'loss': 0.13948775827884674}
{'epoch': 1, 'step': 850, 'loss': 0.19221387803554535}
{'epoch': 1, 'step

Iteration:   0%|          | 0/5001 [00:00<?, ?it/s]

{'epoch': 2, 'step': 0, 'loss': 0.23489829897880554}
{'epoch': 2, 'step': 50, 'loss': 0.09463536739349365}
{'epoch': 2, 'step': 100, 'loss': 0.03274427726864815}
{'epoch': 2, 'step': 150, 'loss': 0.031769104301929474}
{'epoch': 2, 'step': 200, 'loss': 0.1963762789964676}
{'epoch': 2, 'step': 250, 'loss': 1.0071572065353394}
{'epoch': 2, 'step': 300, 'loss': 0.0292824674397707}
{'epoch': 2, 'step': 350, 'loss': 0.015553206205368042}
{'epoch': 2, 'step': 400, 'loss': 0.0013863713247701526}
{'epoch': 2, 'step': 450, 'loss': 0.3949871063232422}
{'epoch': 2, 'step': 500, 'loss': 0.8535099029541016}
{'epoch': 2, 'step': 550, 'loss': 0.0008604002650827169}
{'epoch': 2, 'step': 600, 'loss': 0.0011603101156651974}
{'epoch': 2, 'step': 650, 'loss': 0.00015256863844115287}
{'epoch': 2, 'step': 700, 'loss': 0.0011077946983277798}
{'epoch': 2, 'step': 750, 'loss': 0.007898322306573391}
{'epoch': 2, 'step': 800, 'loss': 0.009769098833203316}
{'epoch': 2, 'step': 850, 'loss': 0.007279332727193832}
{'

In [ ]:
!pip install transformers datasets accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.0 MB/s 
     |████████████████████████████████| 451 kB 69.2 MB/s 
     |████████████████████████████████| 191 kB 75.0 MB/s 
     |████████████████████████████████| 182 kB 67.8 MB/s 
     |████████████████████████████████| 7.6 MB 55.3 MB/s 
     |████████████████████████████████| 132 kB 67.0 MB/s 
     |████████████████████████████████| 212 kB 76.0 MB/s 
     |████████████████████████████████| 127 kB 72.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
from transformers import BartTokenizerFast, BartForQuestionAnswering, AutoConfig, default_data_collator
from torch.utils.data import DataLoader
from accelerate import Accelerator
from tqdm.auto import tqdm
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%ls

model/


In [ ]:
!fusermount -u drive
!google-drive-ocamlfuse drive

shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
fusermount: bad mount point drive: Transport endpoint is not connected
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
/bin/bash: google-drive-ocamlfuse: command not found


In [ ]:
%cd ./drive/"MyDrive"/"Colab Notebooks"/BERT_DRCD
# %ls ./model/

/content/drive/MyDrive/Colab Notebooks/BERT_DRCD


In [ ]:
from transformers import AutoTokenizer, AutoConfig, BertForQuestionAnswering
tokenizer = AutoTokenizer.from_pretrained('bert-base-chinese')
config = AutoConfig.from_pretrained('./model/new_epoch_2/config.json')
model = BertForQuestionAnswering.from_pretrained('./model/new_epoch_2/pytorch_model.bin', config=config)

In [ ]:
from pathlib import Path
def read_data(path, limit=None):
    path = Path(path)
    with open(path, 'rb') as f:
        data_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
    unanswers = {'text': '', 'answer_start': 0}
    for group in data_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                answer_list = qa['answers']
                if len(answer_list) == 0:
                  contexts.append(context)
                  questions.append(question)
                  answers.append(unanswers)
                else:
                  for answer in answer_list:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)
                  if limit != None and len(contexts) > limit:
                    return contexts, questions, answers
                  
    return contexts, questions, answers

In [ ]:
# eval_contexts, eval_questions, eval_answers = read_data('dev-v2.0.json',2000)
eval_contexts, eval_questions, eval_answers = read_data(eval_path,2000)

In [ ]:
print(eval_contexts[0])
print(eval_questions[0])
print(eval_answers[0])

在歐洲，梵語的學術研究，由德國學者陸特和漢斯雷頓開創。後來威廉·瓊斯發現印歐語系，也要歸功於對梵語的研究。此外，梵語研究，也對西方文字學及歷史語言學的發展，貢獻不少。1786年2月2日，亞洲協會在加爾各答舉行。會中，威廉·瓊斯發表了下面這段著名的言論：「梵語儘管非常古老，構造卻精妙絕倫：比希臘語還完美，比拉丁語還豐富，精緻之處同時勝過此兩者，但在動詞詞根和語法形式上，又跟此兩者無比相似，不可能是巧合的結果。這三種語言太相似了，使任何同時稽考三者的語文學家都不得不相信三者同出一源，出自一種可能已經消逝的語言。基於相似的原因，儘管缺少同樣有力的證據，我們可以推想哥德語和凱爾特語，雖然混入了迥然不同的語彙，也與梵語有著相同的起源；而古波斯語可能也是這一語系的子裔。」
陸特和漢斯雷頓開創了哪一地區對梵語的學術研究？
{'id': '1', 'text': '歐洲', 'answer_start': 1}


In [ ]:
def add_end_idx(answers):
    for answer in answers:
        gold_text = answer['text']
        start_idx = answer['answer_start']
        if gold_text == '':
          end_idx = 0
        else:
          end_idx = start_idx + len(gold_text) # Find end character index of answer in context
        answer['answer_end'] = end_idx

add_end_idx(eval_answers)

In [ ]:
eval_encodings = tokenizer(eval_contexts, eval_questions, truncation=True, padding=True)

In [ ]:
def add_token_positions(encodings,answers):
  start_positions = []
  end_positions = []
  for i in range(len(answers)):
    if answers[i]['answer_start'] == 0 and answers[i]['answer_end'] == 0:
      start_positions.append(0)
      end_positions.append(0)
    else:
      start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
      end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))
    # if None, the answer passage has been truncated
    if start_positions[-1] is None:
      start_positions[-1] = tokenizer.model_max_length
    if end_positions[-1] is None:
      end_positions[-1] = tokenizer.model_max_length
  encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

In [ ]:
print(len(eval_answers))
add_token_positions(eval_encodings, eval_answers)

2002


In [ ]:
import torch
class Dataset(torch.utils.data.Dataset):
  def __init__(self, encodings):
    self.encodings = encodings

  def __getitem__(self, idx):
    return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

  def __len__(self):
    return len(self.encodings.input_ids)

In [ ]:
eval_dataset = Dataset(eval_encodings)

In [ ]:
from transformers import BertTokenizerFast, BertForQuestionAnswering, AutoConfig, default_data_collator
from torch.utils.data import DataLoader
from accelerate import Accelerator
from tqdm.auto import tqdm
import json

In [ ]:
eval_batch_size = 12      # 設定 batch size
data_collator = default_data_collator

eval_dataloader = DataLoader(eval_dataset, collate_fn=data_collator, batch_size=eval_batch_size)

# Initialize the accelerator. We will let the accelerator handle device placement for us in this example.
accelerator = Accelerator()

# Prepare everything with our `accelerator`.
model, eval_dataloader = accelerator.prepare(
    model, eval_dataloader
)

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-chinese')

print("***** Running eval *****")
model.eval()
score = []
pre = []
ref = []
prediction_text = []
answer_text = []
index = 0

for batch in eval_dataloader:
  with torch.no_grad():
    # context = batch['context']
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    start_position = batch['start_positions']
    end_position = batch['end_positions']

    outputs = model(input_ids, attention_mask = attention_mask)
    start_pred = torch.argmax(outputs['start_logits'], dim=1)
    end_pred = torch.argmax(outputs['end_logits'], dim=1)


    for i in range(len(input_ids)):
      # print(tokenizer.decode(input_ids[i]))
      # context=tokenizer.decode(input_ids[i])
      # print('問題:',context[context.find('[SEP]'): context.rfind('[SEP]')])
      pre_text=tokenizer.decode(input_ids[i][start_position[i]: end_position[i]])
      ref_text=tokenizer.decode(input_ids[i][start_pred[i]: end_pred[i]])
      # print('預測:',pre_text)
      # print('答案:',ref_text)
      # print("--------------------------------------")
      prediction_text.append(pre_text)
      answer_text.append(ref_text)
      pre.append({'prediction_text': pre_text, 'id': str(index)})
      ref.append({'answer': {'answer_start': start_pred[i], 'text': ref_text}, 'id': str(index)})

  

    score.append(((start_pred == start_position).sum()/len(start_pred)).item())
    score.append(((end_pred == end_position).sum()/len(end_pred)).item())

    index+=1

score = sum(score)/len(score)

***** Running eval *****


In [ ]:
score

0.402744512960404

In [ ]:
torch.save(model,"./test_model2")

In [ ]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 1.4 MB/s 


In [ ]:
from evaluate import load
exact_match_metric = load("exact_match")
em_results = exact_match_metric.compute(predictions=prediction_text, references=answer_text)
print('em:',em_results)
print(prediction_text)
print(answer_text)


em: {'exact_match': 0.3191808191808192}
['L', 'L', '斯', '斯', ' 究 。', ' 究 。', ' 語 文 獻 的 ', ' 語 文 獻 的 ', '的 ； ', '的 ； ', ' 系 ', ' 系 ', '清 ', '清 ', '用', '用', '0 年', '90 年 ', 'CLS', 'CLS', '島 、 亮', '島 、 亮', ' ', '的 馬 ', 'CLS', 'CLS', ' 與 ', ' 與 ', '、 廚', ' ', '犬 燈 ', '燈 ', ' 花 崗 石 建 造 的 洋 式 燈 ', ' 洋 式 燈 ', ' 例 。', ' 例 。', '艇 ', '艇 ', ' 新 型', ' 新 型', '了 南 ', '南 ', ' 業 ', 'S] ', '有 ', '有 ', '祖 戰 ', '祖 戰 ', ' 學', ' 學', ' ', ' ', ' 村 牛 角 社 區', ' 村 牛 角 社 區', '財 庫 。 馬 ', '財 庫 。 馬 ', ' 本 境 ', ' 本 境 ', ' 庫 戶 ', ' 補 庫 戶 ', 'C', 'C', '、 ', ' 高', '軍 ', ' 光', '政 府 所', '政 府 所', ' ', ' ', '連 ', '連 ', ' ', ' ', '的', '的', ' ', ' ', ' 28', ' 28', '', '日 序', ' 紀 ， 此 為', ' 小 紀 ， 此 為', 'C', 'C', '打 至', '打 至', ' ', ' 明 ', '球 ', '球 ', ' ', ' ', '', '', 'S', 'S', '賽', '賽', ' 賽 ', ' 賽 ', 'NK] 」', 'NK] 」', ' 方 法 ', ' 方 法 ', '離 地 ', '離 地 ', ' 球', ' 球', '', '', '改 良 ， 用 網 代 ', '改 良 ， 用 網 代 ', 'C', 'C', '和', '和', ' ， 使 對 手 難 以 回 球 。 截', ' 截', '1926 ', '1926 ', '運 動 員 組 成 。 最 值 得 稱 道 的 早', '運 動 員 組 成 。 最 值 得 稱 道 的 早',

In [ ]:
f1_metric = evaluate.load("f1")
f1_results = f1_metric.compute(predictions=pre, references=ref)
print('f1:',f1_results)

NameError: ignored

In [ ]:
for i in range(15,25):
  print('\n',i)
  print("true label = ",ref[i])
  print("pred label = ",pre[i])



 15
true label =  {'answer': {'answer_start': tensor(149), 'text': ' 梵 文 ， 其 實 它 只 是 蘭 札 體 的 一 種 無 頭 化 變 體 。 藏 文 、 蒙 文 、 托 忒 蒙 文 、 滿 文 都 有 用 各 自 的 文 字 轉 寫 梵 文 的 體 系 。 1991 年 印 度 人 口 普 查 報 告 了 49, 736 名 流 利 的 梵 語 使 用 者 。 自 從 1990 年 代 ， 對 復 興'}, 'id': '1'}
pred label =  {'prediction_text': '用', 'id': '1'}

 16
true label =  {'answer': {'answer_start': tensor(205), 'text': ' 蒙 文 、 滿 文 都 有 用 各 自 的 文 字 轉 寫 梵 文 的 體 系 。 1991 年 印 度 人 口 普 查 報 告 了 49'}, 'id': '1'}
pred label =  {'prediction_text': '0 年', 'id': '1'}

 17
true label =  {'answer': {'answer_start': tensor(205), 'text': ' 蒙 文 、 滿 文 都 有 用 各 自 的 文 字 轉 寫 梵 文 的 體 系 。 1991 年 印 度 人 口 普 查 報 告 了 49'}, 'id': '1'}
pred label =  {'prediction_text': '90 年 ', 'id': '1'}

 18
true label =  {'answer': {'answer_start': tensor(1), 'text': 'CLS] 馬 祖 列 島 是 隸 屬 中 華 民 國 的 群 島 ， 位 於 臺 灣 海 峽 正 北 方 ， 面 臨 閩 江 口 、 連 江 口 和 羅 源 灣 ， 與 中 國 大 陸 僅 一 水 之 隔 ， 距 中 國 大 陸 最 近 點 約 9. 25 公 里 。 主 要 由 南 '}, 'id': '1'}
pred label =  {'prediction_text': 'CLS', 'id': '1'}

 1

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd /content/drive/"MyDrive"/"Colab Notebooks"
%cd ./BERT_DRCD/

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks
/content/drive/MyDrive/Colab Notebooks/BERT_DRCD


In [ ]:
from transformers import AutoTokenizer
import torch

tokenizer=AutoTokenizer.from_pretrained('bert-base-chinese')
model = torch.load("test_model2")
# model.eval()

In [ ]:
# **撰寫預測程式**
def QA_model(model, context, question):

  input = tokenizer.encode_plus(question, context, return_tensors='pt')
  outputs = model(**input)
  answer_start = torch.argmax(outputs[0])
  answer_end = torch.argmax(outputs[1]) + 1

  answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input['input_ids'][0][answer_start:answer_end]))
  

  return answer

In [ ]:
context = "《哈利波特》（英語：Harry Potter），英國作家J·K·羅琳的奇幻文學系列小說，描寫主角哈利波特在霍格華茲魔法學校7年學習生活中的冒險故事；該系列被翻譯成77種語言，在超過兩百個國家出版，所有版本的總銷售量5億，名列世界上最暢銷小說之列。[1][2][3]美國華納兄弟電影公司把這7集小說改拍成8部電影，前6集各一部，第7集分兩部。哈利波特電影系列是全球史上最賣座的電影系列，總票房收入過77億美元，在多國上映。[4]此外還有位於佛羅里達的主題樂園，與大量衍生作品，使作者羅琳本人在此作品完成後躋身英國最富裕的女性人物之列，超越英國女王伊莉莎白二世。"
question = "哈利波特的作家是誰?"

answer = QA_model(model, context, question)  
print(answer)

In [ ]:
context = "哈利波特的作家是J·K·羅琳"
question = "哈利波特的作家是誰?"

answer = QA_model(model, context, question)  
print(answer)

[UNK] · [UNK] · 羅 琳


In [ ]:
context = "國立中興大學，是位於臺中的一所高等教育機構。中興大學以農業科學、農業經濟學、獸醫、生命科學、轉譯醫學、生醫工程、生物科技、綠色科技等研究領域見長 。近年中興大學與臺中榮民總醫院、彰化師範大學、中國醫藥大學等機構合作，聚焦於癌症醫學、免疫醫學及醫學工程三項領域，將實驗室成果逐步應用到臨床上。興大也與臺中市政府合作，簽訂合作意向書，共同推動數位文化、智慧城市等面相帶動區域發展[6]。"
question = "國立中興大學位於哪裡?"

answer = QA_model(model, context, question)  
print(answer)



臺 中


In [ ]:
context = "國立中興大學簡稱興大，是位於臺中的一所高等教育機構。中興大學以農業科學、農業經濟學、獸醫、生命科學、轉譯醫學、生醫工程、生物科技、綠色科技等研究領域見長 。近年中興大學與臺中榮民總醫院、彰化師範大學、中國醫藥大學等機構合作，聚焦於癌症醫學、免疫醫學及醫學工程三項領域，將實驗室成果逐步應用到臨床上。興大也與臺中市政府合作，簽訂合作意向書，共同推動數位文化、智慧城市等面相帶動區域發展[6]。"
question = "國立中興大學簡稱?"

answer = QA_model(model, context, question)  
print(answer)



興 大
